## Getting the justice bio info
Wikipedia is a great source for summary info of a lot of things. Their lack of javascript is very inviting for webscraping (however, you can't really write a script to iterate through pages on wikipedia since there are thousands of different authors, therefore many ways of presenting the same problem. So the best use for wikipedia is if you need a table or two from a single page). 

In [1]:
import requests
import re 
from bs4 import BeautifulSoup
import pandas as pd

import html5lib
import re
tbls1 = pd.read_html("https://en.wikipedia.org/wiki/List_of_Justices_of_the_Supreme_Court_of_the_United_States")
tbls1 = tbls1[1]
df = pd.DataFrame(tbls1)

In [2]:
df.columns = ["dummy", "name", "State", "lifetime", "service", "ChiefJus", "Retired", "Appointedby", "termination" ]

In [3]:
def lastname(text):
    lastname = text.split(",")
    return(lastname[0])

def service(text):
    service = str(text).split("(")
    return(service[0])



def replacestuff(text):
    pattern = text.find("........-")
    text = str(text)
    text.replace(pattern.group(), "")

    
df["lastname"] = df.name.apply(lastname)
df["serviceyrs"] = df.service.apply(service)

df["lastname"] = df["lastname"].map(lambda x: x.upper())
#df.info()

In [7]:
df.lastname.value_counts().to_csv("cleanedsupjustices.csv")

In [22]:
def start(text):
    text = list(str(text))
    serve1 = text[0:4]
    return ''.join(serve1)

def end(text):
    text = str(text)
    text = text.split(' ')
    text = text[0]
    serve1 = text[-4:]
    serve1 = serve1.replace('sent', 'now')
    return str(serve1)
df['start'] = df.service.apply(start)
df['end'] = df.service.apply(end)

In [24]:
df.to_pickle('justicesummary.pickle')

## We also need the political affiliations of the people who nominated these justices
I didn't include this info for analysis (at this time), I included it because I thought it would be dope to see. This page was hilariously broken up so I wrote a script for each one. If there were any more than there were I would have written a function, but I was already half way through by the time I thought about doing that.

In [36]:
import requests
import re 
from bs4 import BeautifulSoup
import pandas as pd

import html5lib
import re
tbls1 = pd.read_html("https://en.wikipedia.org/wiki/List_of_Presidents_of_the_United_States_by_political_affiliation")
republicans = tbls1[1]
dems = tbls1[2]
waffles = tbls1[3]
whigs = tbls1[4]
federalists = tbls1[5]
nonparty = tbls1[6]
repu = pd.DataFrame(republicans)
demos = pd.DataFrame(dems)
waff = pd.DataFrame(waffles)
whig= pd.DataFrame(whigs)
fed= pd.DataFrame(federalists)
nonparty= pd.DataFrame(nonparty)
repu['party'] = 'republican'
demos['party'] = 'democrat'
waff['party'] = 'waffles'
whig['party'] = 'whigs'
fed['party'] = 'fed'
nonparty['party'] = 'nonpartisan'

In [41]:
#these tables were weird when they came in, this fixed them
repu = repu.reindex(repu.index.drop(0))
demos = demos.reindex(demos.index.drop(0))
waff = waff.reindex(waff.index.drop(0))
whig = whig.reindex(whig.index.drop(0))
fed = fed.reindex(fed.index.drop(0))
nonparty = nonparty.reindex(nonparty.index.drop(0))

In [43]:
pd.concat([repu, demos, waff, whig, fed, nonparty]).to_pickle('politicalparties.pickle')